In [96]:
def simplify(label):
  if label == 'NotNegative' or label == 'Positive':
    return 'Positive'
  if label == 'NotPositive' or label == 'Negative':
    return 'Negative'
  else:
    return 'None'

In [97]:
import json
def load_prediction(fname):
  pred = {}
  with open(fname) as f:
    for line in f:
      line_elem = json.loads(line.strip())
      pred[line_elem['docId']] = {k['holder']+'##'+k['target']: k['sentiment'] for k in line_elem['sentiments']}
  return pred

def load_gold(fname):
  gold = {}
  with open(fname) as f:
    for line in f:
      line_elem = json.loads(line.strip())
      gold[line_elem['docId']] = {k['holder']+'##'+k['target']: simplify(k['sentiment']) for k in line_elem['sentiments']}
  return gold

In [147]:
def f1(p, r):
  if r == 0.:
    return 0.
  return 2 * p * r / float(p + r)
def f1_micro(true_and_prediction):
  num_examples = len(true_and_prediction)
  num_predicted_labels = 0.
  num_true_labels = 0.
  num_correct_labels = 0.
  pred_example_count = 0.
  for true_labels, predicted_labels in true_and_prediction:
    if predicted_labels:
      pred_example_count += 1
    num_predicted_labels += len(predicted_labels)
    num_true_labels += len(true_labels)
    num_correct_labels += len(set(predicted_labels).intersection(set(true_labels)))
  if pred_example_count == 0:
    return num_examples, 0, 0, 0, 0, 0
  precision = num_correct_labels / num_predicted_labels
  recall = num_correct_labels / num_true_labels
  avg_elem_per_pred = num_predicted_labels / pred_example_count
  #return num_examples, pred_example_count, avg_elem_per_pred, precision, recall, f1(precision, recall)
  return '{0:.1f}\t{1:.1f}\t{2:.1f}'.format(precision*100, recall*100, f1(precision, recall)*100)

In [148]:
def compute_f1(pred, gold):
  gold_pred_list = []
  total_doc_count = 0
  for doc_id, gold_i in gold.items():
    if not doc_id in pred:
      continue
    total_doc_count += 1
    for holder_target, pred_ii in pred[doc_id].items():
      gold_pred = (gold_i.get(holder_target, 'None'), pred_ii)
      gold_pred_list.append(gold_pred)
  print(gold_pred_list[:10])
  pos_list = [(int(g=='Positive'), int(p=='Positive')) for (g, p) in gold_pred_list]
  pos_list = [([] if g == 0 else [1], [] if p == 0 else [1]) for (g, p) in pos_list]
  neg_list = [(int(g=='Negative'), int(p=='Negative')) for (g, p) in gold_pred_list]
  neg_list = [([] if g == 0 else [1], [] if p == 0 else [1]) for (g, p) in neg_list]
  print("Positive", f1_micro(pos_list))
  print("Negative", f1_micro(neg_list))
  print(total_doc_count)

In [158]:
pred_model = load_prediction('/Users/echoi/Project/document-e2e-sent/prediction/dev.json')
gold_ann = load_gold('/Users/echoi/Project/document-e2e-sent/data/annotation/acl_total.json')
compute_f1(pred_model, gold_ann)

[('None', 'None'), ('None', 'Negative'), ('None', 'None'), ('None', 'Negative'), ('None', 'Negative'), ('Negative', 'Negative'), ('None', 'None'), ('Negative', 'Negative'), ('None', 'Negative'), ('Negative', 'Negative')]
Positive 44.4	39.2	41.6
Negative 17.4	30.4	22.1
38
